In [1]:
from scrapy.crawler import CrawlerRunner, CrawlerProcess
import scrapy
from crochet import setup, wait_for
import os
import json

In [2]:
urls = [
        'https://www.americanas.com.br/busca/celular?limit=50',
        'https://www.americanas.com.br/busca/televisao?limit=50'
       ]

In [3]:
class AmericanasSetSpider(scrapy.Spider):
    name = "americanas-produtos"

    file_links = []
    
    def start_requests(self):
        for i, link in enumerate(self.start_urls):
            #import pdb;pdb.set_trace()
            name_file = link.split("/")[-1].split("?")[0]
            self.file_links.append(open(f"{name_file}.json",'w'))
            self.file_links[i].write("{\"products\":[")
            request = scrapy.Request(link, callback=self.parse, cb_kwargs=dict(index_file=i))
            yield request
            
    def parse(self, response, index_file):
        
        for link_product in response.xpath('//div[re:test(@class, "inStockCard__Wrapper-sc-1ngt5zo-0 iRvjrG")]//a[1]/@href').getall():
           # import pdb;pdb.set_trace()

            complete_link = f"https://www.americanas.com.br{link_product}"
            #file_links.write(complete_link)

            #NAME_SELECTOR = 'ul data-testid'
            yield response.follow(complete_link, callback=self.parseproduct, cb_kwargs=dict(index_file=index_file))
            
 
    def parseproduct(self, response, index_file):
        #import pdb;pdb.set_trace()
        title = response.xpath('//h1[re:test(@class, "product-title__Title-sc-1hlrxcw-0 jyetLr")]/text() ').get()
        title = title.replace("\"", "\\\"")
        description = response.xpath('//div[re:test(@class, "src__Description-sc-13f3i2j-2 bqnMru")]//text() ').getall()
        description = "".join(description).replace("\n", " ").replace("\"", "\\\"")
        self.file_links[index_file].write(f"{{\"title\":\"{title}\", \"description\":\"{description}\"}},")


In [4]:
SETTINGS = {
            'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',         
            } 

In [5]:
process = CrawlerProcess(SETTINGS)
process.crawl(AmericanasSetSpider,start_urls= urls)
process.start()

2021-11-08 22:21:26 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-11-08 22:21:26 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.3 (default, Jul  2 2020, 16:21:59) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 2.9.2, Platform Linux-5.11.0-38-generic-x86_64-with-glibc2.10
2021-11-08 22:21:26 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-11-08 22:21:26 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
2021-11-08 22:21:26 [scrapy.extensions.telnet] INFO: Telnet Password: d877c6fed6526d98
2021-11-08 22:21:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsag

2021-11-08 22:21:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/134444740?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=602607380c07044266a07d27&buyboxToken=smartbuybox-acom-v2-025f4378-87d2-4136-a506-8500bad591d6-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/televisao?limit=50)
2021-11-08 22:21:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3234384115?pfm_carac=celular&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=60a25e7752131c3c8106f535&buyboxToken=smartbuybox-acom-v2-98e39701-02b5-4ebd-9453-ede452987b19-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/celular?limit=50)
2021-11-08 22:21:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3069509067?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=608acb6252131c3c81e2ab21&buyboxToken=smartbuybo

2021-11-08 22:21:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/1508411046?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=5e84f47079bf8430cbbfa8e6&buyboxToken=smartbuybox-acom-v2-e8afd1f6-6819-4e5d-9d2a-c86d54e152dd-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/televisao?limit=50)
2021-11-08 22:21:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/1515649301?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=5e70d46d79bf8430cb40699a&buyboxToken=smartbuybox-acom-v2-31bee3c7-4819-4c7f-b69b-f89446e7805f-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/televisao?limit=50)
2021-11-08 22:21:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3280464600?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=60d9d36052131c3c81ff2c49&buyboxToken=smart

2021-11-08 22:21:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3280455586?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=60afcf5552131c3c81642ee3&buyboxToken=smartbuybox-acom-v2-6462c143-969b-414d-bd8b-85db3f3b8474-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/televisao?limit=50)
2021-11-08 22:21:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3069506106?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=608acb2152131c3c81e23044&buyboxToken=smartbuybox-acom-v2-ff725968-00a2-4cb2-be70-4168dec1d48d-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/televisao?limit=50)
2021-11-08 22:21:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3280822551?pfm_carac=televisao&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6136384b09c351890db9fb8c&buyboxToken=smart

2021-11-08 22:21:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/3485484971?pfm_carac=celular&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6107bc7852131c3c816b96e1&buyboxToken=smartbuybox-acom-v2-a75d37b2-29ec-444c-a6ba-027097f55936-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/celular?limit=50)
2021-11-08 22:21:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/2655734451?pfm_carac=celular&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=6006c8f10c07044266e90bbd&buyboxToken=smartbuybox-acom-v2-ce6bcec5-ff67-4236-bc6b-7248fadbd449-2021-11-08%2022:21:27-0300> (referer: https://www.americanas.com.br/busca/celular?limit=50)
2021-11-08 22:21:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.americanas.com.br/produto/2891924281?pfm_carac=celular&pfm_page=search&pfm_pos=grid&pfm_type=search_page&offerId=607054030c07044266b61f3a&buyboxToken=smartbuybox-aco

In [6]:
def json_correction(start_urls):
    for i, link in enumerate(start_urls):
            #import pdb;pdb.set_trace()
            name_file = link.split("/")[-1].split("?")[0]
            
            with open(f"../data/{name_file}.json", 'rb+') as filehandle:
                filehandle.seek(-1, os.SEEK_END)
                filehandle.truncate()
                filehandle.write(b"]}")

In [7]:
json_correction(urls)

In [9]:
json.load(open("../data/celular.json",'rb'))

{'products': [{'title': "Smartphone Samsung Galaxy M32 128GB 4G Wi-Fi Tela 6,4'' Dual Chip 6GB RAM Câmera Quadrupla + Selfie 20MP - Azul",
   'description': ''},
  {'title': "Smartphone Motorola Moto G60s 128GB 4G Wi-Fi Tela 6,8'' Dual Chip 6GB RAM Câmera Quádrupla + Selfie 16MP - Verde",
   'description': 'O  Smartphone Motorola Moto G60s chegou para surpreender os apaixonados por tecnologia.   Conta com o design atraente e perfeito para você segurar com apenas uma mão. Sua tela é LCD IPS 6.8", ideal para assistir vídeos, jogar e acessar as suas redes sociais preferidas. Um modelo completo.  Outro ponto que chama a atenção é o seu processador Helio G95 (2,0 GHz Octa-Core), que permite que o smartphone seja ainda mais ágil e você possa aproveitar ao máximo. Conte com muito espaço para adicionar fotos, vídeos e muitos arquivos.   O apaixonados por fotos vão adorar a câmera quádrupla traseira de 64MP + 8MP + 5MP + 2MP , que contam com resolução incrível para tirar fotos tanto de dia como